In [180]:
import pandas as pd
import lib.vision as vision

from sample import Sample

In [181]:
labels = pd.read_csv("dataset/labels.csv")

In [182]:
labels.drop(columns=["label_name", "image_width", "image_height"], inplace=True)
labels.rename({"bbox_x": "x1", "bbox_y": "y1"}, axis=1, inplace=True)

In [183]:
labels["x2"] = labels["x1"] + labels["bbox_width"]
labels["y2"] = labels["y1"] + labels["bbox_height"]

In [184]:
labels.drop(columns=["bbox_width", "bbox_height"], inplace=True)

In [185]:
labels = labels[["image_name", "x1", "y1", "x2", "y2"]]

In [186]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  1103 non-null   object
 1   x1          1103 non-null   int64 
 2   y1          1103 non-null   int64 
 3   x2          1103 non-null   int64 
 4   y2          1103 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 43.2+ KB


In [187]:
train_results = pd.read_csv("out/train_result.csv", header=None, names=labels.columns)
val_results = pd.read_csv("out/val_result.csv", header=None, names=labels.columns)

In [188]:
estimates = pd.concat([train_results, val_results])

In [189]:
estimates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 973 entries, 0 to 187
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  973 non-null    object 
 1   x1          962 non-null    float64
 2   y1          962 non-null    float64
 3   x2          962 non-null    float64
 4   y2          962 non-null    float64
dtypes: float64(4), object(1)
memory usage: 45.6+ KB


In [193]:
ground_truths = [Sample(s[0], s[1], s[2], s[3], s[4]) for s in labels.to_numpy()]
estimates = [Sample(s[0], s[1], s[2], s[3], s[4]) for s in estimates.to_numpy()]

In [196]:
iou_threshold = 0.4

In [197]:
true_positives = []

for g_sample in ground_truths:
    estimate_samples = [
        e_sample for e_sample in estimates if e_sample.image_name == g_sample.image_name
    ]
    for e_sample in estimate_samples:
        iou = vision.get_IoU(e_sample.bbox.points, g_sample.bbox.points)
        if iou > 0 and iou >= iou_threshold:
            true_positives.append(e_sample)

In [198]:
# everything that was missed in the ground truth sample
false_negatives = [
    truth_sample for truth_sample in ground_truths if truth_sample not in true_positives
]

In [199]:
tp = len(true_positives)
fn = len(false_negatives)
fp = len(estimates) - tp

In [200]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = (2 * precision * recall) / (precision + recall)

In [201]:
print(precision, recall, f1)

0.68448098663926 0.3764838892029395 0.4857768052516412
